# SNCF - TGV regularity visualization

## Data import, cleaning and transformation

In [53]:
import pandas as pd
import numpy as np
from datetime import datetime
from collections import defaultdict

## Read CSV
df = pd.read_csv("data/regularite-mensuelle-tgv.csv", encoding="utf8", sep=';')

## Drop NaN values
df = df.dropna(subset=df.columns.drop('Commentaires'))

## Create additional column with city set 
df["cities"] = list(map(lambda a, b: frozenset([a, b]), df["Départ"], df["Arrivée"]))


### Identify causes for train disturbances in the observations

In [54]:
disturbance = {"obs_person": ['personne','voiture','tracteur','vol ','incendie','colis','alerte','pompier','police','bagages'],
               "obs_animal": ['animal','animaux','chevreuil','oiseau','sanglier','vaches'],
               "obs_technical": ['problème','défaillance','défaut','disjonction','dérangements','panne','rupture','cassé',
                                'caténaire','incident','aiguillage'],
               "obs_malicious": ['malveillance'],
               "obs_construction": ['travaux'],
               "obs_strike": ['grève','social','manifestants'],
               "obs_weather": ['intempérie','précipitation','inondation','chaleur','arbre','branchage','neige','foudre','orage']}

def find_word(words, sentence):
    # this function returns 1 if one of the word in the list is found in the given sentence.
    # it returns 0 otherwise
    count = 0
    if str(sentence) == 'nan':
        return count
    else:
        for word in words:
            if sentence.find(word) == -1:
                pass
            else:
                count += 1
        return min(count,1)  

# check for each disturbance family if the observation contains related words
for key,value in disturbance.items():
    df[key] = list(map(lambda a: find_word(value,a),df["Commentaires"]))

## Train regularity statistics per railway section

In [ ]:
# Groupby Date and Cities
g1 = df.groupby(["Date", "cities"]).sum()
g1 = g1.reset_index()
g1["Régularité"] = 1 - 1.0*(g1["Nombre de trains annulés"] + g1["Nombre de trains en retard à l'arrivée"])\
    /g1["Nombre de trains programmés"]
g1["year"] = list(map(lambda x: int(x[0:4]), g1.Date))
g1["month"] = list(map(lambda x: int(x[5:]), g1.Date))
g1["datetime"]=pd.to_datetime(g1.Date)


# train_lines dictionnary binds train lines names with the rail sections that are concerned. This dictionnary is then used to
# aggregate the total number of trains running on a specific rail section.

train_lines = defaultdict()
train_lines[frozenset(['NICE VILLE', 'PARIS LYON'])] = ['S1','S1_1','S1_1_1','S1_1_1_1','S1_1_1_1_2', 'S1_1_1_1_2_1', 
                                                        'S1_1_1_1_2_1_2', 'S1_1_1_1_2_1_2_1', 'S1_1_1_1_2_1_2_1_1', 
                                                        'S1_1_1_1_2_1_2_1_1_1']
train_lines[frozenset(['PARIS LYON', 'TOULON'])] = ['S1','S1_1','S1_1_1','S1_1_1_1','S1_1_1_1_2', 'S1_1_1_1_2_1', 
                                                    'S1_1_1_1_2_1_2', 'S1_1_1_1_2_1_2_1', 'S1_1_1_1_2_1_2_1_1']
train_lines[frozenset(['MARSEILLE ST CHARLES', 'PARIS LYON'])] = ['S1','S1_1','S1_1_1','S1_1_1_1','S1_1_1_1_2', 'S1_1_1_1_2_1', 
                                                                  'S1_1_1_1_2_1_2', 'S1_1_1_1_2_1_2_1']
train_lines[frozenset(['AIX EN PROVENCE TGV', 'PARIS LYON'])] = ['S1','S1_1','S1_1_1','S1_1_1_1','S1_1_1_1_2', 'S1_1_1_1_2_1', 
                                                                 'S1_1_1_1_2_1_2']
train_lines[frozenset(['AVIGNON TGV', 'PARIS LYON'])] = ['S1','S1_1','S1_1_1','S1_1_1_1','S1_1_1_1_2', 'S1_1_1_1_2_1']
train_lines[frozenset(['PARIS LYON', 'VALENCE ALIXAN TGV'])] = ['S1','S1_1','S1_1_1','S1_1_1_1','S1_1_1_1_2']
train_lines[frozenset(['PARIS MONTPARNASSE', 'TOURS'])] = ['A1','A1_2']
train_lines[frozenset(['CHAMBERY CHALLES LES EAUX', 'PARIS LYON'])] = ['S1','S1_1','S1_1_1','S1_1_1_1','S1_1_1_1_3']
train_lines[frozenset(['BELLEGARDE (AIN)', 'PARIS LYON'])] = ['S1','S1_1','S1_1_1','S1_1_1_1','S1_1_1_1_3','S1_1_1_1_3_3']
train_lines[frozenset(['ANNECY', 'PARIS LYON'])] = ['S1','S1_1','S1_1_1','S1_1_1_1','S1_1_1_1_3','S1_1_1_1_3_2']
train_lines[frozenset(['METZ', 'PARIS EST'])] = ['E1','E1_1']
train_lines[frozenset(['BORDEAUX ST JEAN', 'PARIS MONTPARNASSE'])] = ['A1','A1_2','A1_2_1','A1_2_1_2','A1_2_1_2_1']
train_lines[frozenset(['LE CREUSOT MONTCEAU MONTCHANIN', 'PARIS LYON'])] = ['S1','S1_1']
train_lines[frozenset(['DUNKERQUE', 'PARIS NORD'])] = ['N1','N1_3','N1_3_1']
train_lines[frozenset(['PARIS MONTPARNASSE', 'TOULOUSE MATABIAU'])] = ['A1','A1_2','A1_2_1','A1_2_1_2','A1_2_1_2_1',
                                                                       'A1_2_1_2_1_1']
train_lines[frozenset(['LILLE', 'PARIS NORD'])] = ['N1','N1_2']
train_lines[frozenset(['LILLE', 'MARSEILLE ST CHARLES'])] = ['N1','N1_2','S1','S1_1','S1_1_1','S1_1_1_1','S1_1_1_1_2', 
                                                             'S1_1_1_1_2_1','S1_1_1_1_2_1_2','S1_1_1_1_2_1_2_1']
train_lines[frozenset(['LYON PART DIEU', 'MARSEILLE ST CHARLES'])] = ['S1_1_1_1_2','S1_1_1_1_2_1','S1_1_1_1_2_1_2',
                                                                      'S1_1_1_1_2_1_2_1']
train_lines[frozenset(['PARIS MONTPARNASSE', 'QUIMPER'])] = ['A1','A1_1','A1_1_1','A1_1_1_3','A1_1_1_3_1']
train_lines[frozenset(['PARIS MONTPARNASSE', 'RENNES'])] = ['A1','A1_1','A1_1_1']
train_lines[frozenset(['ANGERS SAINT LAUD', 'PARIS MONTPARNASSE'])] = ['A1','A1_1','A1_1_2']
train_lines[frozenset(['NANTES', 'STRASBOURG'])] = ['A1','A1_1','A1_1_2','A1_1_2_1','E1','E1_1','E1_1_1']
train_lines[frozenset(['MULHOUSE VILLE', 'PARIS LYON'])] = ['S1','S1_2','S1_2_1','S1_2_1_1']
train_lines[frozenset(['NANTES', 'PARIS MONTPARNASSE'])] = ['A1','A1_1','A1_1_2','A1_1_2_1']
train_lines[frozenset(['LYON PART DIEU', 'MONTPELLIER'])] = ['S1_1_1_1_2','S1_1_1_1_2_1','S1_1_1_1_2_1_1','S1_1_1_1_2_1_1_1']
train_lines[frozenset(['LYON PART DIEU', 'PARIS LYON'])] = ['S1','S1_1','S1_1_1','S1_1_1_1']
train_lines[frozenset(['PARIS MONTPARNASSE', 'ST MALO'])] = ['A1','A1_1','A1_1_1','A1_1_1_1']
train_lines[frozenset(['LYON PART DIEU', 'RENNES'])] = ['A1','A1_1','A1_1_1','S1','S1_1','S1_1_1','S1_1_1_1']
train_lines[frozenset(['MONTPELLIER', 'PARIS LYON'])] = ['S1','S1_1','S1_1_1','S1_1_1_1','S1_1_1_1_2','S1_1_1_1_2_1',
                                                         'S1_1_1_1_2_1_1','S1_1_1_1_2_1_1_1']
train_lines[frozenset(['ANGOULEME', 'PARIS MONTPARNASSE'])] = ['A1','A1_2','A1_2_1','A1_2_1_2']
train_lines[frozenset(['GRENOBLE', 'PARIS LYON'])] = ['S1','S1_1','S1_1_1','S1_1_1_1','S1_1_1_1_3','S1_1_1_1_3_1']
train_lines[frozenset(['PARIS EST', 'STRASBOURG'])] = ['E1','E1_1','E1_1_1']
train_lines[frozenset(['PARIS LYON', 'SAINT ETIENNE CHATEAUCREUX'])] = ['S1','S1_1','S1_1_1','S1_1_1_1','S1_1_1_1_1']
train_lines[frozenset(['DIJON VILLE', 'PARIS LYON'])] = ['S1','S1_2']
train_lines[frozenset(['PARIS MONTPARNASSE', 'ST PIERRE DES CORPS'])] = ['A1','A1_2']
train_lines[frozenset(['NANCY', 'PARIS EST'])] = ['E1','E1_1']
train_lines[frozenset(['NIMES', 'PARIS LYON'])] = ['S1','S1_1','S1_1_1','S1_1_1_1','S1_1_1_1_2','S1_1_1_1_2_1','S1_1_1_1_2_1_1']
train_lines[frozenset(['PARIS MONTPARNASSE', 'POITIERS'])] = ['A1','A1_2','A1_2_1']
train_lines[frozenset(['LA ROCHELLE VILLE', 'PARIS MONTPARNASSE'])] = ['A1','A1_2','A1_2_1','A1_2_1_1']
train_lines[frozenset(['LILLE', 'LYON PART DIEU'])] = ['N1','N1_2','S1','S1_1','S1_1_1','S1_1_1_1']
train_lines[frozenset(['PARIS MONTPARNASSE', 'VANNES'])] = ['A1','A1_1','A1_1_1','A1_1_1_3']
train_lines[frozenset(['ARRAS', 'PARIS NORD'])] = ['N1','N1_3']
train_lines[frozenset(['PARIS LYON', 'PERPIGNAN'])] = ['S1','S1_1','S1_1_1','S1_1_1_1','S1_1_1_1_2','S1_1_1_1_2_1',
                                                         'S1_1_1_1_2_1_1','S1_1_1_1_2_1_1_1','S1_1_1_1_2_1_1_1_1']
train_lines[frozenset(['LE MANS', 'PARIS MONTPARNASSE'])] = ['A1','A1_1']
train_lines[frozenset(['MACON LOCHE', 'PARIS LYON'])] = ['S1','S1_1','S1_1_1']
train_lines[frozenset(['DOUAI', 'PARIS NORD'])] = ['N1','N1_1']
train_lines[frozenset(['BESANCON FRANCHE COMTE TGV', 'PARIS LYON'])] = ['S1','S1_2','S1_2_1']
train_lines[frozenset(['BREST', 'PARIS MONTPARNASSE'])] = ['A1','A1_1','A1_1_1','A1_1_1_2']
train_lines[frozenset(['PARIS EST', 'REIMS'])] = ['E1']
train_lines[frozenset(['LAVAL', 'PARIS MONTPARNASSE'])] = ['A1','A1_1']

# assign the different rail sections list to each dataframe entry
g1["line_section"] = list(map(lambda x: train_lines[x], g1.cities))

# duplicate rows for each rail section. set line_section value to line section name
g2 = pd.DataFrame()
for ii in range(len(g1)):
    for jj in range(len(g1['line_section'][ii])):
        row = g1.iloc[ii,:].copy()
        row.loc['line_section'] = g1.loc[ii,'line_section'][jj]
        g2 = g2.append(row,ignore_index=True)

# Groupby line_section
g3 = g2.groupby(["Date", "line_section"]).sum()
g3 = g3.reset_index()
g3["regularity"] = 1 - 1.0*(g3["Nombre de trains annulés"] + g3["Nombre de trains en retard à l'arrivée"])\
    /g3["Nombre de trains programmés"]
g3["year"] = list(map(lambda x: int(x[0:4]), g3.Date))
g3["month"] = list(map(lambda x: int(x[5:]), g3.Date))
g3["datetime"]=pd.to_datetime(g3.Date)
g3.head()

# export to tsv file
g3.to_csv('data/regularity.tsv', sep='\t', columns=['Date','year','month','line_section','regularity'], index=False)

In [114]:
g3.head()

,Date,line_section,Nombre de trains annulés,Nombre de trains ayant circulé,Nombre de trains en retard à l'arrivée,Nombre de trains programmés,Régularité,month,year,datetime
0,2011-09,A1,0,10752,806,10752,0.925037,9,2011,2011-09-01
1,2011-09,A1_1,0,6023,398,6023,0.933920,9,2011,2011-09-01
2,2011-09,A1_1_1,0,2391,139,2391,0.941865,9,2011,2011-09-01
3,2011-09,A1_1_1_1,0,114,1,114,0.991228,9,2011,2011-09-01
4,2011-09,A1_1_1_2,0,372,16,372,0.956989,9,2011,2011-09-01


## List of comments per train line per month

In [56]:
'''
# Read CSV
df2 = pd.read_csv("data/regularite-mensuelle-tgv.csv", encoding="utf8", sep=';')

# Drop NaN values
df2 = df.dropna()

# Drop non-used columns
df2 = df2.drop(["Axe", "Nombre de trains programmés","Nombre de trains ayant circulé", 
               "Nombre de trains annulés","Nombre de trains en retard à l'arrivée"],1)
'''
# export to tsv file
df.to_csv('data/comments.tsv', sep='\t', columns=['Date','Départ','Arrivée','Commentaires','regularity','obs_person','obs_animal',
                                                  'obs_weather','obs_malicious','obs_construction','obs_strike','obs_technical'], 
          index=False, encoding="utf8")


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5895 entries, 0 to 5899
Data columns (total 18 columns):
Date                                      5895 non-null object
Axe                                       5895 non-null object
Départ                                    5895 non-null object
Arrivée                                   5895 non-null object
Nombre de trains programmés               5895 non-null float64
Nombre de trains ayant circulé            5895 non-null float64
Nombre de trains annulés                  5895 non-null float64
Nombre de trains en retard à l'arrivée    5895 non-null float64
Régularité                                5895 non-null float64
Commentaires                              1105 non-null object
cities                                    5895 non-null object
obs_person                                5895 non-null int64
obs_animal                                5895 non-null int64
obs_weather                               5895 non-null int64
obs_malic